# Choral Explanations on Alkemio

At the [TIP knowledge base](https://alkem.io/tip/knowledge-base) on [Alkemio](https://welcome.alkem.io/) we apply [Choral Explanations](https://hapgood.us/2016/05/13/choral-explanations/) to develop knowledge commons in a Q&A format. In this project we explore new possibilities for navigation and visualisation.

## Copyright and license

### Code

Copyright: sander.dijkhuis@cleverbase.com 2024; Licensed under the EUPL.

### Example content

See content source.

In [126]:
import json
import os
import re
import shutil
import urllib
from datetime import datetime
from urllib.request import Request
from IPython.display import display, Markdown

os.environ["no_proxy"] = "*"

Use the [GraphQL Playground](https://alkem.io/graphql) to develop queries.

In [158]:
endpoint = "https://alkem.io/api/private/graphql"
reader = open("queries.graphql", "r")
queries = reader.read()
reader.close()

In [76]:
def query(operation, variables):
    payload = {
        "query": queries,
        "operationName": operation,
        "variables": variables
    }
    headers = {
        "Content-Type": "application/json"
    }
    body = json.dumps(payload).encode("utf-8")
    response = urllib.request.urlopen(Request(endpoint, body, headers))
    return json.loads(response.read().decode("utf-8"))

In [160]:
result = query("questions", { "spaceNameId": "tip" })

In [168]:
questions = []

def strip_license(markdown):
    return re.sub(r".*Bijdragen zijn gelicenseerd onder.*\[.*CC BY 4\.0.*\]\(https:\/\/creativecommons\.org\/licenses\/by\/4\.0\/deed\.nl\).*\..*", "", markdown.strip()).strip()

for question_data in result["data"]["space"]["collaboration"]["callouts"]:
    question = {
        "id": question_data["nameID"],
        "title": question_data["framing"]["profile"]["displayName"],
        "url": question_data["framing"]["profile"]["url"],
        "description_md": strip_license(question_data["framing"]["profile"]["description"]),
        "answers": []
    }
    by = question_data["createdBy"]
    if by != None:
        question["author_name"] = by["profile"]["displayName"]
        question["author_url"] = by["profile"]["url"]
    questions.append(question)
    for answer_data in question_data["contributions"]:
        answer = {
            "id": answer_data["post"]["nameID"],
            "title": answer_data["post"]["profile"]["displayName"],
            "url": answer_data["post"]["profile"]["url"],
            "author_name": answer_data["post"]["createdBy"]["profile"]["displayName"],
            "author_url": answer_data["post"]["profile"]["url"],
            "description_md": answer_data["post"]["profile"]["description"],
            "comments": []
        }
        question["answers"].append(answer)
        for comment_data in answer_data["post"]["comments"]["messages"]:
            comment = {
                "author_name": comment_data["sender"]["profile"]["displayName"],
                "author_url": comment_data["sender"]["profile"]["url"],
                "date": datetime.fromtimestamp(comment_data["timestamp"] / 1000),
                "content_md": comment_data["message"]
            }
            answer["comments"].append(comment)

In [118]:
def link(label, url):
    return f"[🏔️ {label}]({url})"

def linked_title(contribution):
    return link(contribution["title"], contribution["url"])

def linked_author(contribution):
    return link(contribution["author_name"], contribution["author_url"])

def quote(markdown):
    return "".join([f">{line}\n" for line in markdown.splitlines()])

def readable_date(contribution):
    return contribution["date"].strftime("%Y-%m-%d %H:%M UTC")

In [171]:
dir_path = "example/tip"
shutil.rmtree(dir_path)
os.makedirs(dir_path, exist_ok=True)

for question in questions:
    file_path = f"{dir_path}/{question['id']}.md"
    with open(file_path, "w") as file:
        file.write("[🏔️ Alkemio](https://welcome.alkem.io/) › [🏔️ TIP](https://alkem.io/tip/dashboard) › Kennisbank\n")
        file.write(f"# {linked_title(question)}\n")
        if "author_name" in question:
            file.write(f"Oorspronkelijk gevraagd door {linked_author(question)}\n")
        file.write(quote(question["description_md"]))
        if len(question["answers"]) > 0:
            file.write("## Antwoorden\n")
        for answer in question["answers"]:
            file.write(quote(f"### {linked_title(answer)}\n"))
            file.write(quote(f"Oorspronkelijk geantwoord door {linked_author(answer)}\n"))
            file.write(quote(quote(answer["description_md"])))
            if len(answer["comments"]) > 0:
                file.write(quote("#### Reacties\n"))
            for comment in answer["comments"]:
                file.write(quote(quote(f"##### {linked_author(comment)} {readable_date(comment)}\n")))
                file.write(quote(quote(quote(comment["content_md"]))))
        file.write("* * *\n<small>Bijdragen zijn gelicenseerd onder [🌐 CC BY 4.0](https://creativecommons.org/licenses/by/4.0/deed.nl).</small>")
        file.write("\n")